# With TF

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

2023-07-16 19:50:51.646133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_generator= ImageDataGenerator(
    rotation_range= 15, 
    width_shift_range= 0.1, 
    height_shift_range= 0.1,
    horizontal_flip= True,
    vertical_flip= True
)
# test generator without transformation
test_generator= ImageDataGenerator()

train_path= 'dataset/dogs-vs-cat/train'
valid_path= 'dataset/dogs-vs-cat/valid'
test_path= 'dataset/dogs-vs-cat/test'

train_batches= train_generator.flow_from_directory(
    train_path,
    target_size= (224, 224)
)
val_batches= train_generator.flow_from_directory(
    valid_path,
    target_size= (224, 224)
)
test_batches= test_generator.flow_from_directory(
    test_path,
    target_size= (224, 224)
)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [3]:
train_batches.target_size

(224, 224)

In [4]:
train_batches.batch_size

32

In [5]:
base_model= vgg16.VGG16(include_top= False, 
                        weights= 'imagenet', 
                        input_shape= (224, 224, 3),
                        )
base_model.summary()

2023-07-16 19:50:54.854170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 19:50:54.855426: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
# freeze all layers
for layer in base_model.layers:
    layer.trainable= False

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
last_layer= base_model.get_layer(index= -1)
last_layer_output= last_layer.output
last_layer_output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
x= Flatten()(last_layer_output)
x= Dense(64, activation= 'relu', name= 'FC_2')(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(2, activation= 'softmax', name= 'softmax')(x)

new_model= Model(inputs= base_model.input, outputs= x)
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
new_model.compile(loss= 'binary_crossentropy', 
                  optimizer= Adam(learning_rate= 0.0001),
                  metrics= ['accuracy'])

In [11]:
epochs= 10
new_model.fit(train_batches, 
              steps_per_epoch= 4, 
              validation_data= val_batches, 
              validation_steps= 2, 
              epochs= epochs, 
              verbose= 1)

Epoch 1/10
4/4 [==============================] - 25s 7s/step - loss: 0.9653 - accuracy: 0.5859 - val_loss: 1.8455 - val_accuracy: 0.6406
Epoch 2/10
4/4 [==============================] - 24s 7s/step - loss: 0.8077 - accuracy: 0.6562 - val_loss: 1.1999 - val_accuracy: 0.7656
Epoch 3/10
4/4 [==============================] - 24s 7s/step - loss: 0.7608 - accuracy: 0.6415 - val_loss: 1.2327 - val_accuracy: 0.7031
Epoch 4/10
4/4 [==============================] - 24s 7s/step - loss: 0.6050 - accuracy: 0.7500 - val_loss: 0.6844 - val_accuracy: 0.7969
Epoch 5/10
4/4 [==============================] - 22s 6s/step - loss: 0.5048 - accuracy: 0.8019 - val_loss: 0.7684 - val_accuracy: 0.7969
Epoch 6/10
4/4 [==============================] - 24s 7s/step - loss: 0.4107 - accuracy: 0.8594 - val_loss: 0.4394 - val_accuracy: 0.8594
Epoch 7/10
4/4 [==============================] - 21s 6s/step - loss: 0.5175 - accuracy: 0.8302 - val_loss: 0.3964 - val_accuracy: 0.8438
Epoch 8/10
4/4 [==================

In [12]:
test_loss, test_acc= new_model.evaluate(test_batches)
print(f'Test accuracy = {test_acc: .4f} - test loss = {test_loss: 4f}')

15/15 [==============================] - 55s 4s/step - loss: 0.2993 - accuracy: 0.9113
Test accuracy =  0.9113 - test loss =  0.299255


# With Pytorch

In [31]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets

from pathlib import Path
import os
from PIL import Image
import numpy as np

In [36]:
dir_= Path('dataset/dogs-vs-cat/train')
train_set= datasets.ImageFolder(dir_)

dir_= Path('dataset/dogs-vs-cat/valid')
val_set= datasets.ImageFolder(dir_)

dir_= Path('dataset/dogs-vs-cat/test')
test_set= datasets.ImageFolder(dir_)